In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mannwhitneyu, anderson
from statsmodels.stats.multitest import multipletests
from sklearn.utils import resample

# Константы для анализа
ALPHA = 0.05
N_BOOTSTRAPS = 2000

# ==============================================
# 1. ЗАГРУЗКА ДАННЫХ
# ==============================================

# Загрузка данных
df = pd.read_excel('hw_1_ds.xlsx')

# Добавляем расчетные столбцы ДО фильтрации
df['date'] = df['datetime'].dt.date
df['is_test'] = ~df['control_group_flg']
df['bonus_pct'] = np.where(
    df['monetary_amt'] > 0,
    df['bonus_turn_amt'] / df['monetary_amt'] * 100,
    0
)

# Удаляем нулевые транзакции и дубликаты (если нужна строгая уникальность)
df_clean = (
    df[df['transaction_amt'] > 0]
    .drop_duplicates(subset=['customer_id', 'transaction_id'])  # На случай дублей транзакций
    .copy()
)

# ==============================================
# 2. ПОМЕТКА ПЕРВЫХ ПОКУПОК
# ==============================================

# Добавляем ранжирование транзакций по времени для каждого клиента
df_analysis = df_clean.copy()

# Сортировка по клиенту и времени
df_analysis = df_analysis.sort_values(['customer_id', 'datetime'])

# Помечаем первую транзакцию для каждого клиента
df_analysis['transaction_rank'] = df_analysis.groupby('customer_id').cumcount() + 1
df_analysis['is_first_transaction'] = df_analysis['transaction_rank'] == 1

# ==============================================
# 3. ПРОВЕРКА ПЕРЕСЕЧЕНИЙ КЛИЕНТОВ МЕЖДУ ГРУППАМИ
# ==============================================

print("=== АНАЛИЗ ПЕРЕСЕЧЕНИЙ КЛИЕНТОВ ===\n")

# Разделение по акциям
df_7_new = df_analysis[df_analysis['promo'].str.contains('7% на Новых', na=False)].copy()
df_6_otk = df_analysis[df_analysis['promo'].str.contains('6% на Отток', na=False)].copy()

# Выделение групп для акции 7%
test_7_customers = set(df_7_new[~df_7_new['control_group_flg']]['customer_id'].unique())
control_7_customers = set(df_7_new[df_7_new['control_group_flg']]['customer_id'].unique())

# Выделение групп для акции 6%
test_6_customers = set(df_6_otk[~df_6_otk['control_group_flg']]['customer_id'].unique())
control_6_customers = set(df_6_otk[df_6_otk['control_group_flg']]['customer_id'].unique())

# Проверка пересечений для акции 7%
intersection_7_test_control = test_7_customers.intersection(control_7_customers)
intersection_7_test_6 = test_7_customers.intersection(test_6_customers)
intersection_7_control_6_control = control_7_customers.intersection(control_6_customers)

# Проверка пересечений для акции 6%
intersection_6_test_control = test_6_customers.intersection(control_6_customers)

print(f"Акция 7% Новые:")
print(f"  Тестовая группа: {len(test_7_customers)} клиентов")
print(f"  Контрольная группа: {len(control_7_customers)} клиентов")
print(f"  Пересечение тест/контроль: {len(intersection_7_test_control)} клиентов")

print(f"\nАкция 6% Отток:")
print(f"  Тестовая группа: {len(test_6_customers)} клиентов")
print(f"  Контрольная группа: {len(control_6_customers)} клиентов")
print(f"  Пересечение тест/контроль: {len(intersection_6_test_control)} клиентов")

print(f"\nМежакционные пересечения:")
print(f"  Тест 7% ∩ Тест 6%: {len(intersection_7_test_6)} клиентов")
print(f"  Контроль 7% ∩ Контроль 6%: {len(intersection_7_control_6_control)} клиентов")

# Флаг критической проблемы
if len(intersection_7_test_control) > 0 or len(intersection_6_test_control) > 0:
    print(f"\nЕсть клиенты в обеих группах одной акции!")

# ==============================================
# 4. ДЕТАЛИЗИРОВАННЫЙ АНАЛИЗ ПО ГРУППАМ
# ==============================================
print("")
print("=== ДЕТАЛИЗИРОВАННЫЙ АНАЛИЗ ПО ГРУППАМ ===\n")

# Функция для детального анализа группы
def analyze_group_transactions(df_group, group_name):
    total_transactions = len(df_group)
    first_transactions = df_group['is_first_transaction'].sum()
    repeat_transactions = (~df_group['is_first_transaction']).sum()
    
    repeat_pct = (repeat_transactions / total_transactions * 100) if total_transactions > 0 else 0
    
    unique_customers = df_group['customer_id'].nunique()
    
    print(f"{group_name}:")
    print(f"  Уникальных клиентов: {unique_customers}")
    print(f"  Всего транзакций: {total_transactions}")
    print(f"  Первые транзакции: {first_transactions}")
    print(f"  Повторные транзакции: {repeat_transactions} ({repeat_pct:.1f}%)")
    
    return {
        'customers': unique_customers,
        'total_transactions': total_transactions,
        'first_transactions': first_transactions,
        'repeat_transactions': repeat_transactions,
        'repeat_pct': repeat_pct
    }

# Анализ для акции 7% Новые
df_7_new_test = df_7_new[~df_7_new['control_group_flg']]
df_7_new_control = df_7_new[df_7_new['control_group_flg']]

print("Акция 7% Новые:")
stats_7_test = analyze_group_transactions(df_7_new_test, "  Тестовая группа")
stats_7_control = analyze_group_transactions(df_7_new_control, "  Контрольная группа")

# Анализ для акции 6% Отток
df_6_otk_test = df_6_otk[~df_6_otk['control_group_flg']]
df_6_otk_control = df_6_otk[df_6_otk['control_group_flg']]

print("\nАкция 6% Отток:")
stats_6_test = analyze_group_transactions(df_6_otk_test, "  Тестовая группа")
stats_6_control = analyze_group_transactions(df_6_otk_control, "  Контрольная группа")

# Проверка баланса групп
print("\n=== АНАЛИЗ БАЛАНСА ГРУПП ===")
balance_7 = abs(stats_7_test['customers'] - stats_7_control['customers']) / (stats_7_test['customers'] + stats_7_control['customers']) * 100
balance_6 = abs(stats_6_test['customers'] - stats_6_control['customers']) / (stats_6_test['customers'] + stats_6_control['customers']) * 100

print(f"Дисбаланс групп 7% Новые: {balance_7:.1f}%")
print(f"Дисбаланс групп 6% Отток: {balance_6:.1f}%")

=== АНАЛИЗ ПЕРЕСЕЧЕНИЙ КЛИЕНТОВ ===

Акция 7% Новые:
  Тестовая группа: 13378 клиентов
  Контрольная группа: 1474 клиентов
  Пересечение тест/контроль: 0 клиентов

Акция 6% Отток:
  Тестовая группа: 3545 клиентов
  Контрольная группа: 342 клиентов
  Пересечение тест/контроль: 0 клиентов

Межакционные пересечения:
  Тест 7% ∩ Тест 6%: 0 клиентов
  Контроль 7% ∩ Контроль 6%: 0 клиентов

=== ДЕТАЛИЗИРОВАННЫЙ АНАЛИЗ ПО ГРУППАМ ===

Акция 7% Новые:
  Тестовая группа:
  Уникальных клиентов: 13378
  Всего транзакций: 16295
  Первые транзакции: 13378
  Повторные транзакции: 2917 (17.9%)
  Контрольная группа:
  Уникальных клиентов: 1474
  Всего транзакций: 1775
  Первые транзакции: 1474
  Повторные транзакции: 301 (17.0%)

Акция 6% Отток:
  Тестовая группа:
  Уникальных клиентов: 3545
  Всего транзакций: 4622
  Первые транзакции: 3545
  Повторные транзакции: 1077 (23.3%)
  Контрольная группа:
  Уникальных клиентов: 342
  Всего транзакций: 433
  Первые транзакции: 342
  Повторные транзакции: 91 

In [3]:
# ==============================================
# 5. РАСЧЕТ МЕТРИК НА УРОВНЕ КЛИЕНТА (ВКЛЮЧАЯ ВСЕХ УЧАСТНИКОВ АКЦИИ)
# ==============================================
print("=== РАСЧЕТ МЕТРИК НА УРОВНЕ КЛИЕНТА (ВКЛЮЧАЯ ВСЕХ УЧАСТНИКОВ) ===\n")

def enrich_client_metrics_with_zeros(client_metrics_df, current_group_size, total_group_size, group_name):
    """
    Добавляет фиктивные записи клиентов с нулевыми метриками 
    для соответствия общему числу участников акции.
    
    Args:
        client_metrics_df (pd.DataFrame): DataFrame с рассчитанными метриками для клиентов с транзакциями.
        current_group_size (int): Количество клиентов в client_metrics_df (с транзакциями).
        total_group_size (int): Общее количество участников акции по условию задачи.
        group_name (str): Название группы для логирования.
        
    Returns:
        pd.DataFrame: Обогащенный DataFrame с метриками для всех участников.
    """
    missing_customers = total_group_size - current_group_size
    
    if missing_customers > 0:
        print(f"  {group_name}: Добавляем {missing_customers} клиентов без транзакций.")
        # Генерируем уникальные ID для недостающих клиентов
        # Предполагаем, что оригинальные customer_id < 5_000_000, чтобы избежать конфликтов
        max_existing_id = client_metrics_df['customer_id'].max() if not client_metrics_df.empty else 0
        start_fake_id = max(5_000_000, max_existing_id + 1)
        fake_customer_ids = range(start_fake_id, start_fake_id + missing_customers)
        
        # Создаем DataFrame с нулевыми метриками для недостающих клиентов
        fake_clients_df = pd.DataFrame({
            'customer_id': fake_customer_ids,
            'avg_check': 0.0,
            'transaction_count': 0,
            'has_repeat_purchase': 0,
            'made_purchase': 0,  # НОВАЯ МЕТРИКА: 0 для клиентов без транзакций
            'total_bonus': 0.0,
            'total_revenue': 0.0,
            'additional_cost': 0.0
        })
        
        # Объединяем реальные и фиктивные данные
        enriched_client_metrics_df = pd.concat([client_metrics_df, fake_clients_df], ignore_index=True)
    else:
        print(f"  {group_name}: Корректировка не требуется.")
        enriched_client_metrics_df = client_metrics_df.copy()

    print(f"  {group_name}: Итоговое количество клиентов: {len(enriched_client_metrics_df)}")
    # Пересчитываем и выводим средние с учетом всех участников
    print(f"  {group_name}: Средний чек (все участники): {enriched_client_metrics_df['avg_check'].mean():.2f}")
    print(f"  {group_name}: Среднее кол-во транзакций (все участники): {enriched_client_metrics_df['transaction_count'].mean():.2f}")
    print(f"  {group_name}: Доля повторных покупок (все участники): {enriched_client_metrics_df['has_repeat_purchase'].mean()*100:.1f}%")
    # --- ВЫВОД НОВОЙ МЕТРИКИ ---
    print(f"  {group_name}: Доля совершивших покупку (все участники): {enriched_client_metrics_df['made_purchase'].mean()*100:.1f}%")
    # ----------------------------
    print(f"  {group_name}: Средние доп. расходы (все участники): {enriched_client_metrics_df['additional_cost'].mean():.2f}")
    
    return enriched_client_metrics_df

def calculate_client_metrics(df_group, group_name):
    """Расчет метрик на уровне клиента для заданной группы"""
    # Сначала считаем метрики только для клиентов с транзакциями
    client_metrics_with_transactions = df_group.groupby('customer_id').apply(
        lambda x: pd.Series({
            'avg_check': x['transaction_amt'].mean(),
            'transaction_count': len(x),
            'has_repeat_purchase': int(len(x) > 1),
            'made_purchase': 1,  # НОВАЯ МЕТРИКА: 1 для клиентов с транзакциями
            'total_bonus': x['bonus_turn_amt'].sum(),
            'total_revenue': x['monetary_amt'].sum(),
            'additional_cost': (
                (x[x['is_first_transaction']]['bonus_pct'] - 3.0) * 
                x[x['is_first_transaction']]['monetary_amt']
            ).sum() if x['is_first_transaction'].any() else 0,
        })
    ).reset_index()
    
    print(f"{group_name} (только с транзакциями):")
    print(f"  Клиентов с транзакциями: {len(client_metrics_with_transactions)}")
    print(f"  Средний чек: {client_metrics_with_transactions['avg_check'].mean():.2f}")
    print(f"  Среднее кол-во транзакций: {client_metrics_with_transactions['transaction_count'].mean():.2f}")
    print(f"  Доля повторных покупок: {client_metrics_with_transactions['has_repeat_purchase'].mean()*100:.1f}%")
    # --- ВЫВОД НОВОЙ МЕТРИКИ ---
    print(f"  Доля совершивших покупку: {client_metrics_with_transactions['made_purchase'].mean()*100:.1f}%")
    # ----------------------------
    print(f"  Средние доп. расходы: {client_metrics_with_transactions['additional_cost'].mean():.2f}")
    
    return client_metrics_with_transactions

# --- НАЧАЛО ИЗМЕНЕНИЙ ---
# Исходные данные о количестве участников из условия задачи
TOTAL_7_NEW_TEST = 459939
TOTAL_7_NEW_CONTROL = 51100
TOTAL_6_OTK_TEST = 32224
TOTAL_6_OTK_CONTROL = 3576

# Расчет метрик для всех групп (только с транзакциями)
print("Акция 7% Новые:")
client_metrics_7_test_raw = calculate_client_metrics(df_7_new_test, "  Тестовая группа")
client_metrics_7_control_raw = calculate_client_metrics(df_7_new_control, "  Контрольная группа")

print("\nАкция 6% Отток:")
client_metrics_6_test_raw = calculate_client_metrics(df_6_otk_test, "  Тестовая группа")
client_metrics_6_control_raw = calculate_client_metrics(df_6_otk_control, "  Контрольная группа")

# Обогащение метрик фиктивными клиентами
print("\n--- ДОБАВЛЕНИЕ КЛИЕНТОВ БЕЗ ТРАНЗАКЦИЙ ---")
client_metrics_7_test = enrich_client_metrics_with_zeros(
    client_metrics_7_test_raw, len(client_metrics_7_test_raw), TOTAL_7_NEW_TEST, "7% Новые Тест"
)
client_metrics_7_control = enrich_client_metrics_with_zeros(
    client_metrics_7_control_raw, len(client_metrics_7_control_raw), TOTAL_7_NEW_CONTROL, "7% Новые Контроль"
)

client_metrics_6_test = enrich_client_metrics_with_zeros(
    client_metrics_6_test_raw, len(client_metrics_6_test_raw), TOTAL_6_OTK_TEST, "6% Отток Тест"
)
client_metrics_6_control = enrich_client_metrics_with_zeros(
    client_metrics_6_control_raw, len(client_metrics_6_control_raw), TOTAL_6_OTK_CONTROL, "6% Отток Контроль"
)
# --- КОНЕЦ ИЗМЕНЕНИЙ ---

# Сохранение обогащенных метрик для дальнейшего анализа
metrics_storage = {
    '7_new_test': client_metrics_7_test,
    '7_new_control': client_metrics_7_control,
    '6_otk_test': client_metrics_6_test,
    '6_otk_control': client_metrics_6_control
}

=== РАСЧЕТ МЕТРИК НА УРОВНЕ КЛИЕНТА (ВКЛЮЧАЯ ВСЕХ УЧАСТНИКОВ) ===

Акция 7% Новые:
  Тестовая группа (только с транзакциями):
  Клиентов с транзакциями: 13378
  Средний чек: 664.16
  Среднее кол-во транзакций: 1.22
  Доля повторных покупок: 16.5%
  Доля совершивших покупку: 100.0%
  Средние доп. расходы: 1843.48
  Контрольная группа (только с транзакциями):
  Клиентов с транзакциями: 1474
  Средний чек: 623.12
  Среднее кол-во транзакций: 1.20
  Доля повторных покупок: 15.4%
  Доля совершивших покупку: 100.0%
  Средние доп. расходы: 0.05

Акция 6% Отток:
  Тестовая группа (только с транзакциями):
  Клиентов с транзакциями: 3545
  Средний чек: 689.55
  Среднее кол-во транзакций: 1.30
  Доля повторных покупок: 21.8%
  Доля совершивших покупку: 100.0%
  Средние доп. расходы: 1467.73
  Контрольная группа (только с транзакциями):
  Клиентов с транзакциями: 342
  Средний чек: 652.56
  Среднее кол-во транзакций: 1.27
  Доля повторных покупок: 20.2%
  Доля совершивших покупку: 100.0%
  Средние

In [4]:
# ==============================================
# 5. АА ТЕСТ - НА КОНТРОЛЬНЫЕ ГРУППЫ
# ==============================================

from scipy.stats import ttest_ind
import numpy as np

# --- Используем МЕНЬШЕ итераций для A/A теста ---
N_AA_ITERATIONS = 1000  # Количество раз, которое мы будем делить группу и сравнивать
ALPHA = 0.05
# -----------------------------------------------

def run_simple_aa_test(group_data, metric, n_iter=N_AA_ITERATIONS, alpha=ALPHA):
    """
    Проводит A/A тест: делит одну группу пополам и применяет t-тест.
    """
    data = group_data[metric].dropna()
    size = len(data)
    
    if size < 2:
        print(f"Недостаточно данных для метрики '{metric}'")
        return None

    sample_size = size // 2
    p_values = []

    print(f"  Запуск {n_iter} итераций A/A теста (t-тест) для метрики '{metric}'...")
    for i in range(n_iter):
        # Перемешиваем и делим на две части
        shuffled = data.sample(frac=1, random_state=i).reset_index(drop=True)
        g1 = shuffled.iloc[:sample_size]
        g2 = shuffled.iloc[sample_size:sample_size*2]

        # Применяем t-тест
        # equal_var=False - Welch's t-test, более устойчив к неравным дисперсиям
        try:
            # Альтернатива "two-sided" и использование Welch's t-test
            stat, p_val = ttest_ind(g1, g2, equal_var=False, nan_policy='omit')
            p_values.append(p_val)
        except Exception as e:
            # На случай, если t-тест не может быть выполнен (например, одна из групп константа)
            # print(f"  Предупреждение на итерации {i}: {e}")
            # Можно добавить p-value = 1.0 или пропустить
            pass
        
    if not p_values:
         print(f"  Не удалось выполнить ни один t-тест для метрики '{metric}'.")
         return None
         
    # Считаем False Positive Rate
    p_values_array = np.array(p_values)
    fpr = np.mean(p_values_array < alpha)
    print(f"  ✅ Метрика '{metric}': FPR = {fpr:.2%} (ожидаем {alpha:.2%})")
    
    # --- Добавим небольшую проверку ---
    if fpr > alpha * 1.5:
        print(f"     ⚠️  ВНИМАНИЕ: FPR ({fpr:.1%}) ВЫШЕ ожидаемого ({alpha:.1%})!")
    elif fpr < alpha * 0.5:
         print(f"     ⚠️  ВНИМАНИЕ: FPR ({fpr:.1%}) значительно НИЖЕ ожидаемого ({alpha:.1%}). Тест консервативен.")
    else:
         print(f"     ✅  FPR в пределах ожидаемого.")
    # ----------------------------------
    
    return {
        'fpr': fpr,
        'p_values': p_values_array
    }

# --- Запуск ПРОСТОГО A/A теста для контрольных групп ---
print("=== A/A ТЕСТ (t-тест): Контрольная группа 7% Новые ===")
simple_aa_results_7_control = {}
# Тестируем ключевые метрики
for metric in ['avg_check', 'transaction_count', 'has_repeat_purchase', 'total_revenue']:
    simple_aa_results_7_control[metric] = run_simple_aa_test(client_metrics_7_control, metric)

print("\n=== A/A ТЕСТ (t-тест): Контрольная группа 6% Отток ===")
simple_aa_results_6_control = {}
for metric in ['avg_check', 'transaction_count', 'has_repeat_purchase', 'total_revenue']:
    simple_aa_results_6_control[metric] = run_simple_aa_test(client_metrics_6_control, metric)


=== A/A ТЕСТ (t-тест): Контрольная группа 7% Новые ===
  Запуск 1000 итераций A/A теста (t-тест) для метрики 'avg_check'...
  ✅ Метрика 'avg_check': FPR = 5.20% (ожидаем 5.00%)
     ✅  FPR в пределах ожидаемого.
  Запуск 1000 итераций A/A теста (t-тест) для метрики 'transaction_count'...
  ✅ Метрика 'transaction_count': FPR = 5.30% (ожидаем 5.00%)
     ✅  FPR в пределах ожидаемого.
  Запуск 1000 итераций A/A теста (t-тест) для метрики 'has_repeat_purchase'...
  ✅ Метрика 'has_repeat_purchase': FPR = 3.80% (ожидаем 5.00%)
     ✅  FPR в пределах ожидаемого.
  Запуск 1000 итераций A/A теста (t-тест) для метрики 'total_revenue'...
  ✅ Метрика 'total_revenue': FPR = 5.50% (ожидаем 5.00%)
     ✅  FPR в пределах ожидаемого.

=== A/A ТЕСТ (t-тест): Контрольная группа 6% Отток ===
  Запуск 1000 итераций A/A теста (t-тест) для метрики 'avg_check'...
  ✅ Метрика 'avg_check': FPR = 6.10% (ожидаем 5.00%)
     ✅  FPR в пределах ожидаемого.
  Запуск 1000 итераций A/A теста (t-тест) для метрики 'trans

In [5]:
# ==============================================
# 6. ОЦЕНКА МОЩНОСТИ ТЕСТА (КОРРЕКТНАЯ ДЛЯ ДИСПРОПОРЦИИ И БУТСТРЭПА)
# ==============================================

def bootstrap_power_analysis(test_data, control_data, metric, n_iterations=1000, alpha=ALPHA):
    """
    Оценка мощности через бутстрэп с учетом диспропорции групп.
    """
    # Фильтруем данные по метрике
    test_vals = test_data[metric].dropna().values
    control_vals = control_data[metric].dropna().values
    
    # Наблюдаемый эффект
    observed_diff = np.mean(test_vals) - np.mean(control_vals)
    
    # Объединенная выборка (для симуляции H0)
    pooled = np.concatenate([test_vals, control_vals])
    
    # Бутстрэп-симуляция
    significant_count = 0
    
    for _ in range(n_iterations):
        # Симуляция H0 (эффекта нет)
        boot_test = np.random.choice(pooled, size=len(test_vals), replace=True)
        boot_control = np.random.choice(pooled, size=len(control_vals), replace=True)
        
        # Разница в бутстрэп-выборках
        diff = np.mean(boot_test) - np.mean(boot_control)
        
        # Проверка значимости (двусторонний тест)
        if abs(diff) >= abs(observed_diff):
            significant_count += 1
    
    # Эмпирическая мощность
    power = 1 - (significant_count / n_iterations)
    
    return {
        'power': power,
        'observed_diff': observed_diff,
        'test_size': len(test_vals),
        'control_size': len(control_vals),
        'n_iterations': n_iterations
    }

print("=== ОЦЕНКА МОЩНОСТИ ТЕСТА (БУТСТРЭП-МЕТОД) ===\n")

power_analysis_summary = {}

for promo_name, test_data, control_data in [
    ('7%_new', client_metrics_7_test, client_metrics_7_control),
    ('6%_otk', client_metrics_6_test, client_metrics_6_control)
]:
    print(f"--- АКЦИЯ {promo_name} ---")
    print(f"  Размеры групп: Тест={len(test_data)}, Контроль={len(control_data)}")
    
    power_analysis_summary[promo_name] = {}
    
    # Анализируемые метрики
    metrics_for_power = ['avg_check', 'transaction_count', 'has_repeat_purchase', 'total_revenue']
    
    for metric in metrics_for_power:
        result = bootstrap_power_analysis(test_data, control_data, metric, n_iterations=1000)
        
        power_analysis_summary[promo_name][metric] = {
            'power': result['power'],
            'observed_diff': result['observed_diff'],
            'test_size': result['test_size'],
            'control_size': result['control_size']
        }
        
        print(f"{metric:25} | Diff={result['observed_diff']:8.2f} | Мощность={result['power']:.3f}")
        
        if result['power'] < 0.8:
            print(f"       ⚠️  Низкая мощность ({result['power']:.1%})")
        else:
            print(f"       ✅ Хорошая мощность ({result['power']:.1%})")

print("\n✅ Анализ мощности завершен")

=== ОЦЕНКА МОЩНОСТИ ТЕСТА (БУТСТРЭП-МЕТОД) ===

--- АКЦИЯ 7%_new ---
  Размеры групп: Тест=459939, Контроль=51100
avg_check                 | Diff=    1.34 | Мощность=0.886
       ✅ Хорошая мощность (88.6%)
transaction_count         | Diff=    0.00 | Мощность=0.487
       ⚠️  Низкая мощность (48.7%)
has_repeat_purchase       | Diff=    0.00 | Мощность=0.725
       ⚠️  Низкая мощность (72.5%)
total_revenue             | Diff=    1.29 | Мощность=0.870
       ✅ Хорошая мощность (87.0%)
--- АКЦИЯ 6%_otk ---
  Размеры групп: Тест=32224, Контроль=3576
avg_check                 | Diff=   13.45 | Мощность=0.862
       ✅ Хорошая мощность (86.2%)
transaction_count         | Diff=    0.02 | Мощность=0.992
       ✅ Хорошая мощность (99.2%)
has_repeat_purchase       | Diff=    0.00 | Мощность=0.932
       ✅ Хорошая мощность (93.2%)
total_revenue             | Diff=   10.81 | Мощность=0.799
       ⚠️  Низкая мощность (79.9%)

✅ Анализ мощности завершен


In [8]:
# ==============================================
# 7. ПОЛНЫЙ СТАТИСТИЧЕСКИЙ АНАЛИЗ (БУТСТРЭП С БАЛАНСИРОВКОЙ)
# ==============================================

def bootstrap_ab_test_final(test_data, control_data, metric, n_iterations=N_BOOTSTRAPS, sample_size=None, alpha=ALPHA):
    """
    Бутстрэп-анализ для сравнения средних с балансировкой размеров выборок.
    Фокус на наблюдаемой разнице и доверительном интервале.
    """
    # Извлекаем данные по метрике и удаляем NaN
    test_vals = test_data[metric].dropna()
    control_vals = control_data[metric].dropna()

    # Определяем размер выборки для балансировки
    if sample_size is None:
        sample_size = min(len(test_vals), len(control_vals))
    
    # Наблюдаемая разница на полных данных
    observed_diff = test_vals.mean() - control_vals.mean()

    # Список для хранения разниц из бутстрэп-выборок
    bootstrap_diffs = []
    
    # --- Бутстрэп-итерации ---
    for _ in range(n_iterations):
        # Случайная выборка с возвращением из тестовой группы
        boot_test = resample(test_vals, n_samples=sample_size, random_state=None)
        # Случайная выборка с возвращением из контрольной группы
        boot_control = resample(control_vals, n_samples=sample_size, random_state=None)
        
        # Разница средних в бутстрэп-выборках
        diff = boot_test.mean() - boot_control.mean()
        bootstrap_diffs.append(diff)

    # --- Расчет финальных статистик ---
    # Доверительный интервал для разницы (процентили бутстрэп-распределения)
    ci_lower = np.percentile(bootstrap_diffs, 100 * (alpha/2))      # Например, 2.5
    ci_upper = np.percentile(bootstrap_diffs, 100 * (1 - alpha/2)) # Например, 97.5

    # Значимость определяем через доверительный интервал
    is_significant = (ci_lower > 0) or (ci_upper < 0)

    return {
        'observed_diff': observed_diff,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper,
        'significant': is_significant,
        'test_size': len(test_vals),
        'control_size': len(control_vals),
        'sample_size_balanced': sample_size,
        'n_iterations': n_iterations
    }

print("=== ПОЛНЫЙ СТАТИСТИЧЕСКИЙ АНАЛИЗ (БУТСТРЭП С БАЛАНСИРОВКОЙ) ===")

# Все бизнес-метрики (включая новую made_purchase)
business_metrics = [
    'made_purchase', # Новая метрика конверсии
    'avg_check',
    'transaction_count', 
    'total_revenue',
    'has_repeat_purchase',
    'total_bonus',
    'additional_cost'
]

ab_test_results_final = {}

for promo_name, test_data, control_data in [
    ('7%_new', client_metrics_7_test, client_metrics_7_control),
    ('6%_otk', client_metrics_6_test, client_metrics_6_control)
]:
    # Используем минимальный размер группы для балансировки
    equal_size = min(len(test_data), len(control_data))
    
    print(f"\n--- АКЦИЯ {promo_name} (размер групп после балансировки: {equal_size}) ---")
    
    ab_test_results_final[promo_name] = {}
    
    for metric in business_metrics:
        # Простая проверка наличия метрики
        if metric not in test_data.columns or metric not in control_data.columns:
             print(f"{metric:20} | Метрика отсутствует в данных")
             continue
             
        result = bootstrap_ab_test_final(test_data, control_data, metric, sample_size=equal_size, n_iterations=N_BOOTSTRAPS)
        ab_test_results_final[promo_name][metric] = result
        
        # Форматированный вывод
        is_binary_metric = metric in ['made_purchase', 'has_repeat_purchase']
        
        if is_binary_metric:
            test_mean = test_data[metric].mean()
            control_mean = control_data[metric].mean()
            print(f"{metric:20} | Конверсия: Тест={test_mean:.1%} vs Контроль={control_mean:.1%}")
            print(f"                  | Разница: {result['observed_diff']:.3f} (95% CI: [{result['ci_lower']:.3f}, {result['ci_upper']:.3f}])")
        else:
            print(f"{metric:20} | Разница: {result['observed_diff']:8.2f} (95% CI: [{result['ci_lower']:7.2f}, {result['ci_upper']:7.2f}])")
        
        # Выводим звездочку, если эффект значим (CI не включает 0)
        significance_marker = "⭐" if result['significant'] else ""
        if significance_marker:
            print(f"                  | {significance_marker}")
        
        if result['significant']:
            direction = "больше" if result['observed_diff'] > 0 else "меньше"
            print(f"                  ✅ Эффект статистически значим (среднее в тесте {direction})")
        else:
            # Комментарий о мощности для поведенческих метрик
            if metric in ['avg_check', 'transaction_count', 'has_repeat_purchase', 'total_revenue']:
                power = power_analysis_summary.get(promo_name, {}).get(metric, {}).get('power', np.nan)
                if not np.isnan(power) and power < 0.3: 
                     print("                  ⚠️  Эффект не значим (низкая мощность теста)")
                else:
                     print("                  ⚠️  Эффект не значим")
            # Для метрик, напрямую связанных с акцией, ожидаем значимость
            elif metric in ['total_bonus', 'additional_cost']:
                 print("                  ⚠️  Эффект не значим (проверить данные/логику)")
            # Для made_purchase тоже ожидаем эффект
            elif metric == 'made_purchase':
                 print("                  ⚠️  Эффект не значим")
            else:
                 print("                  ⚠️  Эффект не значим")

=== ПОЛНЫЙ СТАТИСТИЧЕСКИЙ АНАЛИЗ (БУТСТРЭП С БАЛАНСИРОВКОЙ) ===

--- АКЦИЯ 7%_new (размер групп после балансировки: 51100) ---
made_purchase        | Конверсия: Тест=2.9% vs Контроль=2.9%
                  | Разница: 0.000 (95% CI: [-0.002, 0.002])
                  ⚠️  Эффект не значим
avg_check            | Разница:     1.34 (95% CI: [  -1.10,    3.60])
                  ⚠️  Эффект не значим
transaction_count    | Разница:     0.00 (95% CI: [  -0.00,    0.00])
                  ⚠️  Эффект не значим
total_revenue        | Разница:     1.29 (95% CI: [  -0.95,    3.41])
                  ⚠️  Эффект не значим
has_repeat_purchase  | Конверсия: Тест=0.5% vs Контроль=0.4%
                  | Разница: 0.000 (95% CI: [-0.000, 0.001])
                  ⚠️  Эффект не значим
total_bonus          | Разница:     0.57 (95% CI: [   0.47,    0.68])
                  | ⭐
                  ✅ Эффект статистически значим (среднее в тесте больше)
additional_cost      | Разница:    53.62 (95% CI: [  48.50,

In [9]:
# ==============================================
# 7. ПОЛНЫЙ СТАТИСТИЧЕСКИЙ АНАЛИЗ (БУТСТРЭП С БАЛАНСИРОВКОЙ) - ОДНОСТОРОННИЙ
# ==============================================

def bootstrap_ab_test_final_one_sided(test_data, control_data, metric, n_iterations=N_BOOTSTRAPS, sample_size=None, alpha=ALPHA):
    """
    Односторонний бутстрэп-анализ для сравнения средних с балансировкой.
    Проверяет H1: среднее_тест > среднее_контроль (разница > 0).
    """
    # Извлекаем данные по метрике и удаляем NaN
    test_vals = test_data[metric].dropna()
    control_vals = control_data[metric].dropna()

    # Определяем размер выборки для балансировки
    if sample_size is None:
        sample_size = min(len(test_vals), len(control_vals))
    
    # Наблюдаемая разница на полных данных
    observed_diff = test_vals.mean() - control_vals.mean()

    # Список для хранения разниц из бутстрэп-выборок
    bootstrap_diffs = []
    
    # --- Бутстрэп-итерации ---
    for _ in range(n_iterations):
        boot_test = resample(test_vals, n_samples=sample_size, random_state=None)
        boot_control = resample(control_vals, n_samples=sample_size, random_state=None)
        diff = boot_test.mean() - boot_control.mean()
        bootstrap_diffs.append(diff)

    # --- Расчет финальных статистик ---
    # Доверительный интервал для разницы 
    # Для одностороннего теста (H1: diff > 0) используем односторонний CI [lower_bound, +inf)
    # Но для вывода покажем стандартный двусторонний CI для наглядности
    ci_lower = np.percentile(bootstrap_diffs, 100 * (alpha))      # Например, 5% для alpha=0.05
    ci_upper = np.percentile(bootstrap_diffs, 100 * (1 - alpha/2)) # Например, 97.5%

    # --- Односторонний расчет p-value ---
    # H1: разница > 0. Проверяем, насколько часто бутстрэп-разницы <= 0
    # p_value = P(boot_diff <= 0 | H0: разница <= 0)
    # Но на практике считаем долю бутстрэп-разностей <= 0
    # Если эта доля мала, то H0 (разница <= 0) отвергается в пользу H1 (разница > 0)
    p_value_one_sided = np.mean(np.array(bootstrap_diffs) <= 0)
    
    # Значимость для одностороннего теста H1: разница > 0
    is_significant_positive = p_value_one_sided < alpha

    return {
        'observed_diff': observed_diff,
        'ci_lower_one_sided': ci_lower, # Нижняя граница одностороннего CI
        'ci_upper': ci_upper,           # Верхняя граница (для наглядности)
        'p_value_one_sided': p_value_one_sided,
        'significant_positive': is_significant_positive,
        'test_size': len(test_vals),
        'control_size': len(control_vals),
        'sample_size_balanced': sample_size,
        'n_iterations': n_iterations
    }

print("=== ПОЛНЫЙ СТАТИСТИЧЕСКИЙ АНАЛИЗ (ОДНОСТОРОННИЙ БУТСТРЭП С БАЛАНСИРОВКОЙ) ===")

# Все бизнес-метрики (включая новую made_purchase)
business_metrics = [
    'made_purchase',
    'avg_check',
    'transaction_count', 
    'total_revenue',
    'has_repeat_purchase',
    'total_bonus',
    'additional_cost'
]

ab_test_results_final_one_sided = {}

for promo_name, test_data, control_data in [
    ('7%_new', client_metrics_7_test, client_metrics_7_control),
    ('6%_otk', client_metrics_6_test, client_metrics_6_control)
]:
    equal_size = min(len(test_data), len(control_data))
    print(f"\n--- АКЦИЯ {promo_name} (размер групп после балансировки: {equal_size}) ---")
    
    ab_test_results_final_one_sided[promo_name] = {}
    
    for metric in business_metrics:
        if metric not in test_data.columns or metric not in control_data.columns:
             print(f"{metric:20} | Метрика отсутствует в данных")
             continue
             
        result = bootstrap_ab_test_final_one_sided(test_data, control_data, metric, sample_size=equal_size, n_iterations=N_BOOTSTRAPS)
        ab_test_results_final_one_sided[promo_name][metric] = result
        
        # Форматированный вывод
        is_binary_metric = metric in ['made_purchase', 'has_repeat_purchase']
        
        if is_binary_metric:
            test_mean = test_data[metric].mean()
            control_mean = control_data[metric].mean()
            print(f"{metric:20} | Конверсия: Тест={test_mean:.1%} vs Контроль={control_mean:.1%}")
            print(f"                  | Разница: {result['observed_diff']:.3f}")
            print(f"                  | 95% Односторонний CI (нижняя граница): [{result['ci_lower_one_sided']:.3f}, +∞)")
        else:
            print(f"{metric:20} | Разница: {result['observed_diff']:8.2f}")
            print(f"                  | 95% Односторонний CI (нижняя граница): [{result['ci_lower_one_sided']:7.2f}, +∞)")
        
        # Выводим звездочку, если эффект значим (H1: разница > 0)
        significance_marker = "⭐" if result['significant_positive'] else ""
        if significance_marker:
            print(f"                  | {significance_marker}")
        
        if result['significant_positive']:
            print(f"                  ✅ Эффект статистически значим (среднее в тесте больше)")
        else:
            if metric in ['avg_check', 'transaction_count', 'has_repeat_purchase', 'total_revenue']:
                power = power_analysis_summary.get(promo_name, {}).get(metric, {}).get('power', np.nan)
                if not np.isnan(power) and power < 0.3: 
                     print("                  ⚠️  Эффект не значим (низкая мощность теста)")
                else:
                     print("                  ⚠️  Эффект не значим")
            elif metric in ['total_bonus', 'additional_cost']:
                 print("                  ⚠️  Эффект не значим (проверить данные/логику)")
            elif metric == 'made_purchase':
                 print("                  ⚠️  Эффект не значим")
            else:
                 print("                  ⚠️  Эффект не значим")

=== ПОЛНЫЙ СТАТИСТИЧЕСКИЙ АНАЛИЗ (ОДНОСТОРОННИЙ БУТСТРЭП С БАЛАНСИРОВКОЙ) ===

--- АКЦИЯ 7%_new (размер групп после балансировки: 51100) ---
made_purchase        | Конверсия: Тест=2.9% vs Контроль=2.9%
                  | Разница: 0.000
                  | 95% Односторонний CI (нижняя граница): [-0.002, +∞)
                  ⚠️  Эффект не значим
avg_check            | Разница:     1.34
                  | 95% Односторонний CI (нижняя граница): [  -0.60, +∞)
                  ⚠️  Эффект не значим
transaction_count    | Разница:     0.00
                  | 95% Односторонний CI (нижняя граница): [  -0.00, +∞)
                  ⚠️  Эффект не значим
total_revenue        | Разница:     1.29
                  | 95% Односторонний CI (нижняя граница): [  -0.59, +∞)
                  ⚠️  Эффект не значим
has_repeat_purchase  | Конверсия: Тест=0.5% vs Контроль=0.4%
                  | Разница: 0.000
                  | 95% Односторонний CI (нижняя граница): [-0.000, +∞)
                  ⚠️  Эфф

**Результаты для поведенческих и ключевых метрик:**

| Акция      | Метрика               | Наблюдаемая разница | 95% ДИ                    | Стат. значимость (CI не включает 0) |
| :--------- | :-------------------- | :------------------ | :------------------------ | :---------------------------------- |
| **7% Новые** | `made_purchase` (конверсия) | 0.000               | [-0.002, 0.002]           | ❌ Нет                              |
|            | Средний чек           | +1.34               | [-1.09, 3.60]             | ❌ Нет                              |
|            | Кол-во транзакций     | +0.00               | [-0.00, 0.00]             | ❌ Нет                              |
|            | Доля повторных покупок| +0.000              | [-0.000, 0.001]           | ❌ Нет                              |
|            | Выручка на клиента    | +1.29               | [-0.87, 3.40]             | ❌ Нет                              |
| **6% Отток** | `made_purchase` (конверсия) | +0.014              | [0.001, 0.028]            | ✅ Да                               |
|            | Средний чек           | +13.45              | [-4.73, 41.24]            | ❌ Нет                              |
|            | Кол-во транзакций     | +0.02               | [0.00, 0.04]              | ✅ Да                               |
|            | Доля повторных покупок| +0.005              | [-0.002, 0.011]           | ❌ Нет                              |
|            | Выручка на клиента    | +10.81              | [-6.65, 39.17]            | ❌ Нет                              |


**Результаты для поведенческих и ключевых метрик в одностороннем тесте:**

| Акция      | Метрика               | Наблюдаемая разница | 95% Односторонний ДИ (нижняя граница) | Стат. значимость (H1: разница > 0) |
| :--------- | :-------------------- | :------------------ | :------------------------------------ | :--------------------------------- |
| **7% Новые** | `made_purchase` (конверсия) | 0.000               | -0.002                                | ❌ Нет                             |
|            | Средний чек           | +1.34               | -0.60                                 | ❌ Нет                             |
|            | Кол-во транзакций     | +0.00               | -0.00                                 | ❌ Нет                             |
|            | Доля повторных покупок| +0.000              | -0.000                                | ❌ Нет                             |
|            | Выручка на клиента    | +1.29               | -0.59                                 | ❌ Нет                             |
| **6% Отток** | `made_purchase` (конверсия) | +0.014              | +0.003                                | ✅ Да                              |
|            | Средний чек           | +13.45              | -1.44                                 | ❌ Нет                             |
|            | Кол-во транзакций     | +0.02               | +0.00                                 | ✅ Да                              |
|            | Доля повторных покупок| +0.005              | -0.001                                | ❌ Нет                             |
|            | Выручка на клиента    | +10.81              | -4.30                                 | ❌ Нет                             |
